In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time
from collections import Counter
import numpy as np

import nltk
nltk.data.path

['/home/ec2-user/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/share/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [4]:
import random

## 1. Import and Preprocess Data

In [5]:
# Define a text preprocessor (lemmatizer)
def my_preprocessor(text):
    doc=nlp(text.lower())
    lemmas=[token.lemma_ for token in doc if not token.is_punct | token.is_space]
    texts_out=" ".join(lemmas)
    return texts_out

In [6]:
# Import expanded reg sentences
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/allRegSentsExpand.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              822737 non-null  object
 1   RegSentsExpand  822737 non-null  object
dtypes: object(2)
memory usage: 12.6+ MB
None


In [ ]:
# Convert reg sentences to list
regSentsExpand=df_regSentsExpand['RegSentsExpand'].tolist()
print(len(regSentsExpand), regSentsExpand[0])

In [ ]:
# Examples
print(regSentsExpand[4031])
print(my_preprocessor(regSentsExpand[4031]))

In [ ]:
# Preprocess all expanded reg sentences
regSentsExpand_lemmatized=[my_preprocessor(sent) for sent in regSentsExpand]

In [ ]:
# Examples
print(len(regSentsExpand_lemmatized), regSentsExpand_lemmatized[0])

In [ ]:
print(regSentsExpand[1],regSentsExpand_lemmatized[1])

In [ ]:
# # Save preprocessed expanded reg sentences
# with open('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/regSentsExpand_lemmatized', 'wb') as fp:
#     pickle.dump(regSentsExpand_lemmatized, fp)

## 2. Match Noun Chunks

In [ ]:
# Import preprocessed expanded reg sentences
with open ('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/regSentsExpand_lemmatized', 'rb') as fp:
    regSentsExpand_lemmatized = pickle.load(fp)
print(len(regSentsExpand_lemmatized), regSentsExpand_lemmatized[0])

In [5]:
# Use all unique noun chunks in rule titles
with open ('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/ruleTitleUniqueNounChunks', 'rb') as fp:
    nounchunks = pickle.load(fp)
print(len(nounchunks),nounchunks[0:20])

37735 ['$ 3 immigration user fee', '-PRON- 2014 2018 greenhouse gas emission standard and fuel efficiency standard', '-PRON- acetate', '-PRON- affiliate', '-PRON- approach', '-PRON- associated privacy act exemption', '-PRON- be portable tank', '-PRON- beneficiary', '-PRON- catcher vessel', '-PRON- character', '-PRON- chemical mixture', '-PRON- child act', '-PRON- community access', '-PRON- computer assist execution system linkage', '-PRON- conduct', '-PRON- correspondent bank', '-PRON- customer', '-PRON- customer requirement', '-PRON- dependent', '-PRON- employee']


In [ ]:
# Compile a new re pattern with all noun chunks
pattern=re.compile(r"\b"+r"\b|\b".join(map(re.escape, nounchunks))+r"\b")

In [ ]:
# Match noun chunks in all expanded reg sentences
start_time = time.time()

nounchunk_match=[]
nounchunk_match_words=[]
for sent in regSentsExpand_lemmatized:
    match_words=[]
    match=0
    find=pattern.findall(sent)
    if len(find)>0:
        match_words=find
        match=len(find)
    nounchunk_match.append(match)
    nounchunk_match_words.append(match_words)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Examples of matched noun chunks
print(len(nounchunk_match), len(nounchunk_match_words))
print(nounchunk_match_words[-1], nounchunk_match[-1])

In [ ]:
print(df_regSentsExpand.head())

In [ ]:
print(df_regSentsExpand[df_regSentsExpand['NounChunksMatch']>0].info())

In [ ]:
# Export results: matched noun chunks in expanded reg sentences
df_regSentsExpand['NounChunksMatch']=nounchunk_match
df_regSentsExpand['NounChunkMatchWords']=nounchunk_match_words
df_regSentsExpand.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')

## 3. Noun Chunk Occurences

In [5]:
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           822737 non-null  object
 1   RegSentsExpand               822737 non-null  object
 2   NounChunksMatch              822737 non-null  int64 
 3   NounChunkMatchWords          822737 non-null  object
 4   NounChunkMatchWordsFiltered  822737 non-null  object
 5   NounChunkMatchFiltered       822737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.7+ MB
None


In [6]:
# Remove duplicated articles
IDs_nodup=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/IDs_no_duplicates.csv')
print(IDs_nodup.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   ID      788516 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB
None


In [7]:
df_regSentsExpand['ID']=df_regSentsExpand['ID'].astype('int64')
df_regSentsExpand=IDs_nodup.merge(df_regSentsExpand,on='ID',how='left').reset_index(drop=True)
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           788516 non-null  int64 
 1   RegSentsExpand               788516 non-null  object
 2   NounChunksMatch              788516 non-null  int64 
 3   NounChunkMatchWords          788516 non-null  object
 4   NounChunkMatchWordsFiltered  788516 non-null  object
 5   NounChunkMatchFiltered       788516 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 36.1+ MB
None


In [ ]:
# All unique macthed noun chunks and occurences from expanded reg sentences
allMatchWords=[]
for list in df_regSentsExpand['NounChunkMatchWords']:
    allMatchWords=allMatchWords+list
print(len(allMatchWords))

In [ ]:
print(allMatchWords[100])

In [ ]:
allMatchWordsCount=Counter(allMatchWords)
print(allMatchWordsCount)

In [14]:
df_MatchWords = pd.DataFrame(allMatchWordsCount.items(),columns = ['Noun Chunks','Occurences'])
print(df_MatchWords.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358 entries, 0 to 11357
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Noun Chunks  11358 non-null  object
 1   Occurences   11358 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 177.6+ KB
None


In [15]:
df_MatchWords=df_MatchWords.sort_values('Occurences',ascending=False).reset_index(drop=True)
print(df_MatchWords.head())

          Noun Chunks  Occurences
0            new york       65836
1      new regulation       29880
2       united states       28376
3  federal regulation       22168
4         los angeles       19997


In [16]:
# Export noun chunk occurences
df_MatchWords.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunkOccurences3.csv',index=False)

## 4. Remove General Terms after Human Auditing

In [9]:
# Remove general terms from matched noun chunks in expanded reg sentences
df_MatchWordsRemove=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunkOccurences3_Remove.csv')
print(df_MatchWordsRemove.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Noun Chunks  713 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB
None


In [10]:
# Noun chunks to be removed ("general terms")
matchwords_remove=df_MatchWordsRemove['Noun Chunks'].tolist()
print(matchwords_remove[0:20])

['1 capital', '1 filing', '10 gallon', '10 guideline', '10 specie', '10 usc', '10,000 year', '100 foot', '100 revision', '103 regulation', '11 plant', '11 usc', '12 cfr', '12 specie', '12,500 pound', '120 volt', '13 case', '13 plant', '14,000 pound', '15 cfr']


In [ ]:
# Reg sections with all matched noun chunks
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')
print(df_regSentsExpand.info())

In [12]:
# Remove general terms from all matched noun chunks
df_regSentsExpand['NounChunkMatchWordsFiltered']=np.nan
df_regSentsExpand['NounChunkMatchFiltered']=0
for i in range(0,len(df_regSentsExpand['NounChunkMatchWords'])):
    df_regSentsExpand['NounChunkMatchWordsFiltered'][i]=[w for w in df_regSentsExpand['NounChunkMatchWords'][i] 
                                                         if w not in matchwords_remove]
    df_regSentsExpand['NounChunkMatchFiltered'][i]=len(df_regSentsExpand['NounChunkMatchWordsFiltered'][i])
print(df_regSentsExpand.info())

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           822737 non-null  object
 1   RegSentsExpand               822737 non-null  object
 2   NounChunksMatch              822737 non-null  int64 
 3   NounChunkMatchWords          822737 non-null  object
 4   NounChunkMatchWordsFiltered  822737 non-null  object
 5   NounChunkMatchFiltered       822737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.7+ MB
None


In [13]:
print(df_regSentsExpand[df_regSentsExpand['NounChunkMatchFiltered']>0].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502740 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           502740 non-null  object
 1   RegSentsExpand               502740 non-null  object
 2   NounChunksMatch              502740 non-null  int64 
 3   NounChunkMatchWords          502740 non-null  object
 4   NounChunkMatchWordsFiltered  502740 non-null  object
 5   NounChunkMatchFiltered       502740 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 26.8+ MB
None


In [14]:
df_regSentsExpand.head()

,ID,RegSentsExpand,NounChunksMatch,NounChunkMatchWords,NounChunkMatchWordsFiltered,NounChunkMatchFiltered
0,294326637,"""Deregulation of natural gas will lower your h...",1,[natural gas],[natural gas],1
1,294308147,Because rates can be adjusted only after a pri...,3,"[price change, price change, federal rule]","[price change, price change, federal rule]",3
2,294323196,A federal appeals court yesterday upheld the N...,2,"[nuclear regulatory commission, nuclear power ...","[nuclear regulatory commission, nuclear power ...",2
3,294311708,A man less secure in his prospects might have ...,0,[],[],0
4,294262284,Legislation which would reduce employers' unem...,4,"[unemployment compensation, tax liability, cre...","[unemployment compensation, tax liability, cre...",4
5,294234319,US/WORLD NEWS BRIEFS The Nuclear Regulator...,2,"[nuclear regulatory commission, low power]",[nuclear regulatory commission],1
6,294323101,"The plant closed Dec. 10, 1983 for its regular...",5,"[nuclear regulatory commission, nuclear regula...","[nuclear regulatory commission, nuclear regula...",5
7,294205735,"Meanwhile, the Federal Aviation Administration...",1,[safety regulation],[safety regulation],1
8,294222941,"As more and more children recover, fewer and f...",0,[],[],0
9,294261564,"They are Sens. William R. Keating (D-Sharon), ...",3,"[public safety, state administration, governme...","[public safety, state administration, governme...",3


In [19]:
# Check some examples
for i in range(0,100):
    print(df_regSentsExpand['NounChunkMatchWords'][i])
    print(df_regSentsExpand['NounChunkMatchWordsFiltered'][i],'\n')

['natural gas']
['natural gas'] 

['price change', 'price change', 'federal rule']
['price change', 'price change', 'federal rule'] 

['nuclear regulatory commission', 'nuclear power plant']
['nuclear regulatory commission', 'nuclear power plant'] 

[]
[] 

['unemployment compensation', 'tax liability', 'credit card', 'interest rate']
['unemployment compensation', 'tax liability', 'credit card', 'interest rate'] 

['nuclear regulatory commission', 'low power']
['nuclear regulatory commission'] 

['nuclear regulatory commission', 'nuclear regulatory commission', 'radiation protection', 'radiation exposure', 'construction worker']
['nuclear regulatory commission', 'nuclear regulatory commission', 'radiation protection', 'radiation exposure', 'construction worker'] 

['safety regulation']
['safety regulation'] 

[]
[] 

['public safety', 'state administration', 'government regulation']
['public safety', 'state administration', 'government regulation'] 

['attorney general']
['attorney gen

In [20]:
df_regSentsExpand.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')